## subsets.jsonの作成

### 3d座標の定義

In [1]:
import os
import sys
import cv2
import json

In [2]:
# キャリブレーションキューブの形状を定義
length = 0.96
width = 0.52
height = 0.96

In [4]:
points = {
    "point_1": [0, 0, 0],
    "point_2": [length, 0, 0],
    "point_3": [length, 0, width],
    "point_4": [0, 0, width],
    "point_5": [0, height, 0],
    "point_6": [length, height, 0],
    "point_7": [length, height, width],
    "point_8": [0, height, width]
}

# JSON形式に変換
markers = []
for name, coords in points.items():
    marker = {
        "name": name,
        "world_position": coords
    }
    markers.append(marker)

# 結果確認
markers_json = {"markers": markers}
markers_json

{'markers': [{'name': 'point_1', 'world_position': [0, 0, 0]},
  {'name': 'point_2', 'world_position': [0.96, 0, 0]},
  {'name': 'point_3', 'world_position': [0.96, 0, 0.52]},
  {'name': 'point_4', 'world_position': [0, 0, 0.52]},
  {'name': 'point_5', 'world_position': [0, 0.96, 0]},
  {'name': 'point_6', 'world_position': [0.96, 0.96, 0]},
  {'name': 'point_7', 'world_position': [0.96, 0.96, 0.52]},
  {'name': 'point_8', 'world_position': [0, 0.96, 0.52]}]}

### 2d座標の取得

In [5]:
# GUIアプリケーション
class ImageApp:
    def __init__(self, img_dir: str, resize_height: int, save_dir: str = os.getcwd()):
        self.img_dir = img_dir
        self.img = None
        self.resize_height = resize_height
        self.subsets = []  # 出力用データリスト
        self.coords = []
        self.rate = None
        self.window_name = "image"
        cv2.namedWindow(self.window_name)
        cv2.moveWindow(self.window_name, 100, 100)

    def onMouse(self, event, x, y, flags, params):
        if event == cv2.EVENT_LBUTTONDOWN:
            org_x, org_y = int(x / self.rate), int(y / self.rate)
            self.coords.append((org_x, org_y))
            self.draw_coordinates(self.coords)
            cv2.imshow(self.window_name, self.img)
            print(self.coords)

    def draw_coordinates(self, coords):
        for coord in coords:
            x, y = coord
            x = int(x * self.rate)
            y = int(y * self.rate)
            cv2.circle(self.img, (x, y), 3, (0, 0, 255), -1)

    def getResizeRate(self, height):
        return self.resize_height / height

    def resize(self, img):
        return cv2.resize(img, None, fx=self.rate, fy=self.rate)

    def run(self):
        img_list = os.listdir(self.img_dir)
        for img_name in img_list:
            print(f"Processing {img_name}")
            img_path = os.path.join(self.img_dir, img_name)
            self.img = cv2.imread(img_path)
            if self.img is None:
                if img_name.endswith(".MOV") or img_name.endswith(".mov"):
                    cap = cv2.VideoCapture(img_path)
                    ret, frame = cap.read()
                    if not ret:
                        print(f"Cannot read {img_name}")
                        continue
                    self.img = frame
                else:
                    continue
            h, w = self.img.shape[:2]
            self.rate = self.getResizeRate(h)
            self.img = self.resize(self.img)
            self.img_copy = self.img.copy()
            cv2.imshow(self.window_name, self.img)
            cv2.setMouseCallback(self.window_name, self.onMouse)

            while True:
                key = cv2.waitKey(0) & 0xFF
                if key == ord("q"):
                    sys.exit("Quit")
                elif key == ord("c"):
                    self.coords = []
                    print(self.coords)
                elif key == ord("p"):  # pop
                    if len(self.coords) > 0:
                        self.coords.pop()
                        self.img = self.img_copy.copy()
                        self.draw_coordinates(self.coords)
                        cv2.imshow(self.window_name, self.img)
                        print(self.coords)
                elif key == ord("s"):
                    for coord in self.coords:
                        # 各クリックごとの座標を辞書形式で保存
                        key_name = f"{img_name}"
                        self.subsets.append({key_name: list(coord)})
                    self.coords = []
                    break

            cv2.destroyAllWindows()
        print("All coordinates data collected")
        output_data = self.subsets
        return output_data

if __name__ == "__main__":
    input_folder = "cube_data"
    resize_height = 1080  # 画像の高さを1080にリサイズ
    app = ImageApp(img_dir=input_folder, resize_height=resize_height)
    subsets_json = app.run()

Processing 3d_points.json
Processing cam1.MOV
[(588, 660)]
[(588, 660), (745, 707)]
[(588, 660)]
[(588, 660), (744, 706)]
[(588, 660), (744, 706), (848, 683)]
[(588, 660), (744, 706)]
[(588, 660), (744, 706), (847, 683)]
[(588, 660), (744, 706), (847, 683), (688, 643)]
[(588, 660), (744, 706), (847, 683)]
[(588, 660), (744, 706), (847, 683), (688, 641)]
[(588, 660), (744, 706), (847, 683)]
[(588, 660), (744, 706), (847, 683), (687, 640)]
[(588, 660), (744, 706), (847, 683)]
[(588, 660), (744, 706), (847, 683), (687, 641)]
[(588, 660), (744, 706), (847, 683), (687, 641), (587, 541)]
[(588, 660), (744, 706), (847, 683), (687, 641), (587, 541), (745, 573)]
[(588, 660), (744, 706), (847, 683), (687, 641), (587, 541)]
[(588, 660), (744, 706), (847, 683), (687, 641), (587, 541), (744, 572)]
[(588, 660), (744, 706), (847, 683), (687, 641), (587, 541)]
[(588, 660), (744, 706), (847, 683), (687, 641), (587, 541), (744, 574)]
[(588, 660), (744, 706), (847, 683), (687, 641), (587, 541)]
[(588, 66

In [6]:
# 変換処理
result = []
for i in range(len(subsets_json) // 2):  # データを半分に分ける
    merged = {}
    merged.update(subsets_json[i])  # 前半部分のデータを追加
    merged.update(subsets_json[i + len(subsets_json) // 2])  # 後半部分のデータを追加
    result.append(merged)

# 結果を確認
subsets_json = {"subsets":result}

In [7]:
output_json = {**subsets_json, **markers_json}

# JSONファイルに保存
output_file = "subsets.json"
with open(output_file, "w") as f:
    json.dump(output_json, f, indent=4)

print(f"JSONデータを {output_file} に保存しました。")

JSONデータを subsets.json に保存しました。
